<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
import requests
import pandas as pd

In [2]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [16]:
toc

['THE TRAGEDY OF ANTONY AND CLEOPATRA',
 'AS YOU LIKE IT',
 'THE COMEDY OF ERRORS',
 'THE TRAGEDY OF CORIOLANUS',
 'CYMBELINE',
 'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK',
 'THE FIRST PART OF KING HENRY THE FOURTH',
 'THE SECOND PART OF KING HENRY THE FOURTH',
 'THE LIFE OF KING HENRY THE FIFTH',
 'THE LIFE OF KING HENRY V',
 'THE SECOND PART OF KING HENRY THE SIXTH',
 'THE THIRD PART OF KING HENRY THE SIXTH',
 'KING HENRY THE EIGHTH',
 'KING JOHN',
 'THE TRAGEDY OF JULIUS CAESAR',
 'THE TRAGEDY OF KING LEAR',
 'LOVE’S LABOUR’S LOST',
 'THE TRAGEDY OF MACBETH',
 'MACBETH',
 'THE MERCHANT OF VENICE',
 'THE MERRY WIVES OF WINDSOR',
 'A MIDSUMMER NIGHT’S DREAM',
 'MUCH ADO ABOUT NOTHING',
 'THE TRAGEDY OF OTHELLO, MOOR OF VENICE',
 'OTHELLO, THE MOOR OF VENICE',
 'KING RICHARD THE SECOND',
 'KING RICHARD THE THIRD',
 'THE TRAGEDY OF ROMEO AND JULIET',
 'THE TAMING OF THE SHREW',
 'THE TEMPEST',
 'THE LIFE OF TIMON OF ATHENS',
 'THE TRAGEDY OF TITUS ANDRONICUS',
 'THE HISTORY OF TROILUS A

In [3]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...


In [18]:
df_toc.shape

(43, 4)

In [8]:
text = " ".join(data)

In [9]:
chars = list(set(text))

In [11]:
len(chars)

104

In [12]:
# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [13]:
# Create the sequence data
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1111146


In [14]:
sequences[0]

[74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 74,
 30,
 74,
 74,
 47,
 1,
 49,
 73,
 74,
 21,
 28,
 52,
 1,
 16,
 56,
 92,
 74,
 54,
 1,
 16,
 28]

In [15]:
[int_char[i] for i in sequences[0]]

[' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '1',
 ' ',
 ' ',
 'F',
 'r',
 'o',
 'm',
 ' ',
 'f',
 'a',
 'i',
 'r',
 'e',
 's',
 't',
 ' ',
 'c',
 'r',
 'e',
 'a']

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN